# Use Pretrained Inception-BatchNorm Network
----

In this example we will demo how to use a pretrained Inception-BatchNorm Network. 

The network is described in 

Ioffe, Sergey, and Christian Szegedy. "Batch normalization: Accelerating deep network training by reducing internal covariate shift." arXiv preprint arXiv:1502.03167 (2015).

For network structure, you can visualize it in [Composite Symbol Demo](composite_symbol.ipynb)

The pre-trained Inception-BatchNorm network is able to be downloaded from:
[http://data.dmlc.ml/mxnet/data/Inception.zip](http://data.dmlc.ml/mxnet/data/Inception.zip)
This model achieves Top-1 Accuracy: 70% and Top-5 Accuracy: 89.9%

Note: This network is trained by using very simple augmentation (random flip + random crop). We will release model with a little bit more augmentation (which achieves better validation score)

In [5]:
%matplotlib inline
import mxnet as mx
import logging
import numpy as np
from skimage import io, transform

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [9]:
dic = mx.nd.load("model/game-0-0010.params")
for k in dic.keys():
    print k
    print dic[k].asnumpy()

arg:fc2_weight
[[ -4.32291269e-01  -1.53905630e-01  -7.41618797e-02   2.00434536e-01
   -1.46718889e-01   1.03650129e+00  -4.98327576e-02   4.87018466e-01
    1.12537943e-01   5.17004311e-01   3.68360549e-01  -2.75213838e-01
    1.85991704e-01  -2.88525194e-01   2.69754887e-01  -3.36150616e-01]
 [  8.66613090e-01  -3.74001525e-02  -4.48411256e-01  -3.28579061e-02
    3.81757677e-01  -4.25438881e-01   1.42480507e-01  -1.77165911e-01
    8.84256209e-04  -1.70755297e-01   1.04132271e+00   3.52574021e-01
   -6.17874324e-01  -1.20580584e-01  -8.29550803e-01   1.26317695e-01]
 [ -8.66450608e-01  -6.68676376e-01   2.29333431e-01   2.59016782e-01
    2.52573103e-01  -1.82353362e-01   3.32577169e-01   2.85807133e-01
    1.26979637e+00  -2.60364801e-01   2.42148519e-01   3.16299796e-01
   -5.48592322e-02   4.22869548e-02   3.34936053e-01   1.25050992e-01]
 [ -2.71868646e-01   7.59455562e-01   9.25505936e-01   3.91237527e-01
   -1.65073201e-01   6.00499175e-02   2.05528975e-01   3.00660700e-01
  

In [11]:
# Load the pre-trained model
prefix = "model/game-0"
num_round = 10
model = mx.model.FeedForward.load(prefix, num_round, ctx=mx.gpu(), numpy_batch_size=1)

In [17]:
import gym
print gym.__file__
env=gym.make('Game-v2')
obs=env.reset()
obs=obs.transpose((2,0,1))
obs=np.expand_dims(obs, axis=0)
print obs.shape
prob = model.predict(obs)[0]
pred = np.argsort(prob)[::-1]

[2016-09-05 09:03:07,944] Making new env: Game-v2


/usr/local/lib/python2.7/dist-packages/gym-0.1.3-py2.7.egg/gym/__init__.pyc
(1, 3, 7, 1)


MXNetError: InferShape Error in conv1: [09:03:07] src/operator/./convolution-inl.h:370: Check failed: ksize_y <= dshape[2] + 2 * param_.pad[0] && ksize_x <= dshape[3] + 2 * param_.pad[1] kernel size exceed input

Then we will show how to use this model to classify image. The image was taken in Notre-Dame Basilica, Montreal.

In [7]:
def PreprocessImage(path, show_img=False):
    # load image
    img = io.imread(path)
    print("Original Image Shape: ", img.shape)
    # we crop image from center
    short_egde = min(img.shape[:2])
    yy = int((img.shape[0] - short_egde) / 2)
    xx = int((img.shape[1] - short_egde) / 2)
    crop_img = img[yy : yy + short_egde, xx : xx + short_egde]
    # resize to 224, 224
    resized_img = transform.resize(crop_img, (224, 224))
    if show_img:
        io.imshow(resized_img)
    # convert to numpy.ndarray
    sample = np.asarray(resized_img) * 256
    # swap axes to make image from (224, 224, 4) to (3, 224, 224)
    sample = np.swapaxes(sample, 0, 2)
    sample = np.swapaxes(sample, 1, 2)
    # sub mean 
    normed_img = sample - mean_img.asnumpy()
    normed_img.resize(1, 3, 224, 224)
    return normed_img

# Get preprocessed batch (single image batch)
batch = PreprocessImage('./DSC01012.JPG', True)
# Get prediction probability of 1000 classes from model
prob = model.predict(batch)[0]
# Argsort, get prediction index from largest prob to lowest
pred = np.argsort(prob)[::-1]
# Get top1 label
top1 = synset[pred[0]]
print("Top1: ", top1)
# Get top5 label
top5 = [synset[pred[i]] for i in range(5)]
print("Top5: ", top5)

NameError: name 'batch' is not defined

To extract feature, it is in similar to steps of [CIFAR-10](cifar-recipe.ipynb)

In [6]:
# get internals from model's symbol
internals = model.symbol.get_internals()
# get feature layer symbol out of internals
fea_symbol = internals["global_pool_output"]
# Make a new model by using an internal symbol. We can reuse all parameters from model we trained before
# In this case, we must set ```allow_extra_params``` to True
# Because we don't need params from FullyConnected symbol
feature_extractor = mx.model.FeedForward(ctx=mx.gpu(), symbol=fea_symbol, numpy_batch_size=1,
                                         arg_params=model.arg_params, aux_params=model.aux_params,
                                         allow_extra_params=True)
# predict feature
global_pooling_feature = feature_extractor.predict(batch)
print(global_pooling_feature.shape)

(1, 1024, 1, 1)
